Задание №1
1. Считать данные из файла
2. Вычислить невязку

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
class Bulk():

    def _to_list(self, line, type):
        return np.array(list(map(type, line.split())), dtype=type)

    def _parse_raw(self, fname):
        txt = ''
        with open(fname, 'r') as f:
            for line in f.readlines():
                if '--' in line:
                    comment = line.index('--')
                    line = line[:comment]
                txt += line

        # split by slashes
        txt = txt.split('/')

        grid_raw = None
        tube_raw = None
        temp_raw = None

        for token in txt:
            token = token.strip()
            if len(token) == 0:
                continue
            elif token.startswith('GRID'):
                grid_raw = self._to_list(token[4:], int)
                #print('parsed GRID')
            elif token.startswith('TUBE'):
                tube_raw = self._to_list(token[4:], float)
                #print('parsed TUBE')
            elif token.startswith('TEMP'):
                temp_raw = self._to_list(token[4:], float)
                #print('parsed TEMP')
            else:
                raise ValueError(f'invalid token: {token[:20]}')

        if grid_raw is None:
            raise ValueError('no GRID')
        if tube_raw is None:
            pass
            #raise ValueError('no TUBE')
        if temp_raw is None:
            raise ValueError('no TEMP')

        return grid_raw, tube_raw, temp_raw

    def _chunks(self, lst, n):
        return [lst[i:i + n] for i in range(0, len(lst), n)]

    def _compute_matr(self, n, sigma):
        mt = np.zeros([n, n])
        mt[0, 0] = 1
        mt[-1, -1] = 1
        for i in range(1, len(mt) - 1):
            mt[i, i - 1] = -sigma[i]
            mt[i, i] = 2 * sigma[i] + 1
            mt[i, i + 1] = -sigma[i]

        mt = np.linalg.inv(mt)
        return mt

    def _compute_sigma(self, tube, i):
        h = self.l / self.n
        tau = self.time[i] - self.time[i - 1]
        assert tau == self.time[1] - self.time[0]
        #print(h, tau)
        ret = tube * tau / h ** 2
        return ret

    def __init__(self, fname, old=False):

        grid_raw, tube_raw, temp_raw = self._parse_raw(fname)

        self.l = grid_raw[0]
        self.n = grid_raw[1]
        self.t = grid_raw[2]
        #tube_raw = np.zeros(self.n)
        if old:
            self.n, self.l = self.l, self.n

        self.tube = tube_raw

        temp_time = self._chunks(temp_raw, self.n + 1)
        if len(temp_time[-1]) != len(temp_time[0]):
            last = list(temp_time[-1])
            while len(last) != len(temp_time[0]):
                last.append(np.nan)
            temp_time[-1] = np.array(last)
        temp_time = np.array(temp_time)
        #print(temp_time)

        self.time = temp_time[:, 0]
        self.temp = temp_time[:, 1:]
        self.temp[self.temp < 0] = np.nan

        self.orig = self.temp.copy()

        self.best_err = 1e9
        self.best_tube = None

    def _plot(self, cmap, mt, title='', ax=None):
        if ax is None:
            fig, ax = plt.subplots(1, 1, figsize=(6, 12))
        ax.set_title(title)
        ax.set_xlabel('bulk')
        ax.set_ylabel('time')
        return ax.imshow(mt, cmap=cmap)

    def _solve(self, a, b, c, d):
        n = len(a)

        ac = a.copy()
        bc = b.copy()
        cc = c.copy()
        dc = d.copy()

        cc[0] /= bc[0]
        dc[0] /= bc[0]

        n -= 1
        for i in range(1, n):
            cc[i] /= bc[i] - ac[i]*cc[i-1]
            dc[i] = (dc[i] - ac[i]*dc[i-1]) / (bc[i] - ac[i]*cc[i-1])

        dc[n] = (dc[n] - ac[n]*dc[n-1]) / (bc[n] - ac[n]*cc[n-1])

        for i in range(n - 1, -1, -1):
            dc[i] -= cc[i] * dc[i+1]

        return dc

    def solve(self, tube=None):

        if tube is None:
            tube = self.tube

        r1 = np.zeros(self.n)
        r2 = np.zeros(self.n)

        for i in range(1, self.n):
            r1[i] = tube[i - 1] + tube[i]

        for i in range(self.n - 1):
            r2[i] = tube[i] + tube[i + 1]

        A = np.zeros(self.n)
        B = np.zeros(self.n)
        C = np.zeros(self.n)

        dt = self.time[1] - self.time[0]
        dx = self.l / self.n

        for i in range(self.n):
            A[i] = -(dt * r1[i]) / (2 * dx * dx)
            B[i] = 1 + (dt*(r1[i]+r2[i]))/(2*dx*dx)
            C[i] = -(dt * r2[i]) / (2 * dx * dx)


        B[0] = 1
        C[0] = 0
        B[-1] = 1
        A[-1] = 0
        A[0] = 0
        C[-1] = 0

        for i in range(1, len(self.temp)):
            self.temp[i] = self._solve(A, B, C, self.temp[i - 1])

        # for i in range(1, len(self.temp)):
        #     sigma = self._compute_sigma(self.tube, i)
        #     A = self._compute_matr(self.n, sigma)

        #     assert A.max() < 2
        #     assert self.temp[i, 0] == self.temp[i - 1, 0]
        #     assert self.temp[i, -1] == self.temp[i - 1, -1]
        #     self.temp[i] = A @ self.temp[i - 1]

    def error(self):
        err = (self.temp - self.orig) ** 2
        err[np.isnan(err)] = 0
        return err.sum()

    def plot_all(self, title=''):
        fig, ax = plt.subplots(1, 3, figsize=(18, 12))
        ax = ax.flatten()
        fig.suptitle(title, fontsize=20)

        p = self._plot('plasma', self.orig, ax=ax[0], title='orig')
        fig.colorbar(p, ax=ax[0], cmap='plasma')

        p = self._plot('plasma', self.temp, ax=ax[1], title='solved')
        fig.colorbar(p, ax=ax[1], cmap='plasma')

        p = self._plot('Reds', (self.temp - self.orig) ** 2, ax=ax[2], title='error')
        fig.colorbar(p, ax=ax[2], cmap='Reds')

        plt.plot()

    def backward(self, maxiter=100):

        for i in range(maxiter):
            tube = np.random.rand(len(self.tube)) * 10

            self.solve(tube)
            err = self.error()

            if err < self.best_err:
                self.best_err = err
                self.best_tube = tube
                print(f'step {i}: err {err}')

    def dump(self):
        with open('out.txt', 'w') as f:
            print(self.n, file=f)
            print(len(self.time), file=f)
            print(self.time[1] - self.time[0], file=f)
            print(self.l / self.n, file=f)
            print(*self.tube, file=f)
            self.orig[np.isnan(self.orig)] = -1
            for i in range(len(self.time)):
                print(*self.orig[i], file=f)
            self.orig[self.orig < 0] = np.nan


In [61]:
bulk.dump()

In [75]:
BLK = 101
bulk = Bulk(f'/content/drive/MyDrive/project/BULK_CHECK/BULK_{BLK}.txt.RE.txt', old=False)
bulk.solve()
print(bulk.error())
# bulk.plot_all(f'bulk {BLK} new')

ValueError: ignored

In [ ]:
bulk.best_err

1000000000.0

In [ ]:
bulk.backward(10000)

step 0: err 4367.899450962751
step 25: err 4245.758133252143
step 61: err 4055.5283785805577
step 70: err 3842.5757878215377
step 127: err 3589.8193642226674
step 129: err 3447.324689821811
step 134: err 3352.1720739222496
step 228: err 3306.8001111377057
step 285: err 3266.7779579635326
step 293: err 3257.95603376093


KeyboardInterrupt: ignored

In [ ]:
print('| N | absolute error | relative error (%) |')
for b in '12345':
    blk = Bulk(f'/content/drive/MyDrive/project/BULK_NEW/BULK_{b}.txt', old=False)
    blk.solve()
    e = blk.error()
    s = np.sum(blk.temp > 0)
    print(f'| {b} | {e:>14.8f} | {e / s * 100:>18.4f} |')


In [ ]:
!pip install cupy

In [ ]:
!pip3 install gnumpy

In [ ]:
import cupy as cp
import numpy as np

x = [3, 4, 5]

x_np = np.array(x)
x_cp = cp.array(x)

l2_np = np.linalg.norm(x_np)
l2_cp = cp.linalg.norm(x_cp)

print("Numpy: ", l2_np)
print("Cupy: ", l2_cp)

In [ ]:
import gnumpy as gpu
import numpy as np

n = 400

a = np.random.uniform(low=0., high=1., size=(n, n)).astype(np.float32)
b = np.random.uniform(low=0., high=1., size=(n, n)).astype(np.float32)

ga = gpu.garray(a)
gb = gpu.garray(b)

ga = ga.dot(gb)
a  = a.dot(b)

print(ga.as_numpy_array(dtype=np.float32) - a)

In [ ]:
import time

# NumPy and CPU Runtime
s = time.time()
x_cpu = np.ones((1000, 1000, 1000))
np_result = np.sqrt(np.sum(x_cpu**2, axis=-1))
e = time.time()
np_time = e - s
print("Time consumed by NumPy: ", np_time)

# CuPy and GPU Runtime
s = time.time()
x_gpu = cp.ones((1000, 1000, 1000))
cp_result = cp.sqrt(cp.sum(x_gpu**2, axis=-1))
e = time.time()
cp_time = e - s
print("\nTime consumed by CuPy: ", cp_time)

# Примерный код


In [ ]:
# TEMP - рассчитываемый массив
# dx - шаг по длине N / lenght
# dt - хз что это
# T - вычисляемый сейчас ряд температуры
#

        TEMP.back().second // данне по температуре с прошлого шага по времени

        double mlt = dt / dx / dx;
        T[0] = TEMP.back().second[0];                   # массив рассчитываемой температуры
        T[sizeX - 1] = TEMP.back().second[sizeX - 1];   # последний элемент

        for (int IS = 1; IS < sizeX - 1; ++IS) {        # рассчитываемый элемент
            double am = 0.5 * (ATEMP[IS] + ATEMP[IS - 1]);  #
            double ap = 0.5 * (ATEMP[IS + 1] + ATEMP[IS]);  #
            // A + C + B = F
            A[IS] = -mlt * am;                              # коэф
            C[IS] = 1 + mlt * (am + ap);                    # коэф
            B[IS] = -mlt * ap;                              # коэф
            if (IS == 1) {                                  # расчет для первого и последнего элемента
                F[IS] = -A[IS] * TEMP.back().second[0];
            }
            else if (IS == sizeX - 2) {
                F[IS] = -B[IS] * TEMP.back().second[sizeX - 1];
            }
            else {
                F[IS] = 0.;
            }
            F[IS] += TEMP.back().second[IS];
        }


        for (int IS = 1; IS < sizeX - 1; ++IS) {
            AL[IS] = 1 == IS ? B[IS] / C[IS] : B[IS] / (C[IS] - AL[IS - 1] * A[IS]);
            BE[IS] = 1 == IS ? F[IS] / C[IS] : (F[IS] - A[IS] * BE[IS - 1]) / (C[IS] - AL[IS - 1] * A[IS]);
        }

        T[sizeX - 2] = BE[sizeX - 2];
        for (size_t IS = sizeX - 3; IS >= 1; --IS) {
            T[IS] = BE[IS] - AL[IS] * T[IS + 1];
        }